In [ ]:
# sample from exponential distribution and try to find the likelihood and 
# posterior plots in the hyp and data spaces. Try plotting 3d plots to find two parameters

In [2]:
# Use bayes rule to compare two model hypotheses.

In [3]:
class Bayesian:
    
    def __init__(pdf):
        self.hyp_pdf = pdf
        self.hyp_params = None
        self.theta = None
    
    
    def likelihood(self, X, min_w_lim=-1000, max_w_lim=1000, nb_of_ws=1000):
        
        # prob(data | params) i.e. prob in hypothesis space
        # assume single param w for now
        
        w_range = np.linspace(min_w_lim, max_w_lim, num=nb_of_ws)
        arr = np.zeros(nb_of_ws)

        for i in range(len(w_range)):
            theta_ = ws1_range[i]
            likelihood = self.pdf(X, theta=theta_)
            arr[i] = likelihood
        likelihood_fn = arr
        return likelihood_fn
    


In [ ]:
exp_pdf = 
bayes = Bayesian()